In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.optim import lr_scheduler
import os
import timm
from timm.data import resolve_model_data_config, create_transform

import shutil

from PIL import Image
from torch.utils.data import Dataset, DataLoader
import pandas as pd

from tqdm import tqdm

In [ ]:
# !unzip -q /content/lnu-deep-learn-1-image-classification-2025.zip -d /content/fast_data


# !rm /content/lnu-deep-learn-1-image-classification-2025.zip

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Використовуємо пристрій: {device}")
if device.type == 'cuda':
    print(f"Назва відеокарти: {torch.cuda.get_device_name(0)}")

data_dir = 'data'
train_dir = os.path.join(data_dir, 'train')

Використовуємо пристрій: cuda
Назва відеокарти: NVIDIA GeForce RTX 3060 Laptop GPU


In [ ]:

# os.chdir('/content')

# print("Поточна папка:", os.getcwd())
# print("Вміст папки:", os.listdir())

In [ ]:

classes_path = os.path.join(data_dir, 'classes.txt')

with open(classes_path, 'r') as f:

    class_names = [line.strip() for line in f.readlines()]

num_classes = len(class_names)

print(f"Кількість класів: {num_classes}")
print(f"Перші 5 класів для прикладу: {class_names[:5]}")

Кількість класів: 100
Перші 5 класів для прикладу: ['ADONIS', 'AFRICAN GIANT SWALLOWTAIL', 'AMERICAN SNOOT', 'AN 88', 'APPOLLO']


In [ ]:
print("Завантажуємо ConvNeXt Base з timm...")


model = timm.create_model(
    "convnext_base.fb_in22k_ft_in1k",  
    pretrained=True,
    num_classes=num_classes
)
cfg = resolve_model_data_config(model)

data_transforms = create_transform(**cfg, is_training=True)


model = model.to(device)


if isinstance(model.get_classifier(), nn.Linear):
    num_ftrs = model.get_classifier().in_features
    print(f"✅ ConvNeXt Base (timm) оновлено. Вхідний розмір фінального шару: {num_ftrs}.")
    print("Новий фінальний шар:", model.get_classifier())
else:
    print("✅ ConvNeXt Base (timm) створено.")

Завантажуємо ConvNeXt Base з timm...
✅ ConvNeXt Base (timm) оновлено. Вхідний розмір фінального шару: 1024.
Новий фінальний шар: Linear(in_features=1024, out_features=100, bias=True)


In [ ]:
train_dir = 'data/train'

train_dataset = datasets.ImageFolder(train_dir, transform=data_transforms)

num_classes = len(train_dataset.classes)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=2,
    pin_memory=True,
    persistent_workers=True
)

print(f"✅ Дані завантажено з: {train_dir}")
print(f"Класів: {num_classes}")
print(f"Зображень для тренування: {len(train_dataset)}")

✅ Дані завантажено з: data/train
Класів: 100
Зображень для тренування: 10911


In [ ]:
for param in model.parameters():
    param.requires_grad = True

In [9]:
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("CUDA кеш очищено.")

CUDA кеш очищено.


In [10]:
torch.set_float32_matmul_precision('high')
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

In [ ]:
criterion = nn.CrossEntropyLoss()
best_acc = 0.0
patience = 5
epochs_without_improvement = 0
checkpoint_path = 'convnext_unfreeze_butterfly_model.pth'


EPOCHS_PHASE_1 = 15


steps_per_epoch = len(train_loader)
total_steps_phase1 = EPOCHS_PHASE_1 * steps_per_epoch


optimizer_1 = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.00001)


scheduler_1 = lr_scheduler.OneCycleLR(
    optimizer_1,
    max_lr=0.01, 
    pct_start=0.3, 
    epochs=EPOCHS_PHASE_1,
    steps_per_epoch=total_steps_phase1,
    anneal_strategy='cos'
)


print("\n--- ФАЗА 1/2: НАВЧАННЯ КЛАСИФІКАТОРА (15 ЕПОХ) ---")

current_optimizer = optimizer_1
current_scheduler = scheduler_1

for epoch in range(EPOCHS_PHASE_1):
    model.train()
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0

    for inputs, labels in tqdm(train_loader, desc=f"Фаза 1 Епоха {epoch+1}"):
        inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)

        current_optimizer.zero_grad()
        with torch.autocast(device_type="cuda", dtype=torch.bfloat16):
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        loss.backward()
        current_optimizer.step()


        current_scheduler.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_preds += labels.size(0)
        correct_preds += (predicted == labels).sum().item()


    epoch_acc = 100 * correct_preds / total_preds
    epoch_loss = running_loss / len(train_loader)


    current_lr = current_optimizer.param_groups[0]['lr']

    print(f"\n🏁 ФАЗА 1, Епоха [{epoch+1}/{EPOCHS_PHASE_1}] | Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.2f}% | LR: {current_lr:.6f}")


    if epoch_acc > best_acc:
        best_acc = epoch_acc
        epochs_without_improvement = 0
        torch.save(model.state_dict(), checkpoint_path)
        print(f"✅ Рекорд! Модель збережено (Acc: {best_acc:.2f}%)")
    else:
        epochs_without_improvement += 1
        print(f"⏹️ Точність не покращилась. Очікування: {epochs_without_improvement}/{patience}")

    if epochs_without_improvement >= patience:
        print(f"🛑 Рання зупинка! Немає покращення {patience} епох.")
        break


print("\n🏆 ФІНАЛ. Тренування завершено!")
print(f"Найкраща точність (збережена): {best_acc:.2f}%")


--- ФАЗА (12 ЕПОХ) ---

--- ФАЗА 1/2: НАВЧАННЯ КЛАСИФІКАТОРА (15 ЕПОХ) ---


Фаза 1 Епоха 1:   0%|          | 0/341 [00:00<?, ?it/s]

Фаза 1 Епоха 1: 100%|██████████| 341/341 [02:07<00:00,  2.67it/s]



🏁 ФАЗА 1, Епоха [1/15] | Loss: 1.0191 | Acc: 74.24% | LR: 0.000405
✅ Рекорд! Модель збережено (Acc: 74.24%)


Фаза 1 Епоха 2: 100%|██████████| 341/341 [01:46<00:00,  3.19it/s]


...


🏁 ФАЗА 1, Епоха [7/15] | Loss: 0.3393 | Acc: 90.06% | LR: 0.000653
⏹️ Точність не покращилась. Очікування: 3/5


Фаза 1 Епоха 8: 100%|██████████| 341/341 [01:51<00:00,  3.06it/s]



🏁 ФАЗА 1, Епоха [8/15] | Loss: 0.3767 | Acc: 89.31% | LR: 0.000729
⏹️ Точність не покращилась. Очікування: 4/5


Фаза 1 Епоха 9: 100%|██████████| 341/341 [01:52<00:00,  3.03it/s]


🏁 ФАЗА 1, Епоха [9/15] | Loss: 0.4138 | Acc: 88.26% | LR: 0.000815
⏹️ Точність не покращилась. Очікування: 5/5
🛑 Рання зупинка! Немає покращення 5 епох.

🏆 ФІНАЛ. Тренування завершено!
Найкраща точність (збережена): 90.59%


In [ ]:
checkpoint_path = 'convnext_unfreeze_butterfly_model.pth'

print("Створення архітектури ConvNeXt (timm) для inference...")
model = timm.create_model(
    "convnext_base.fb_in22k_ft_in1k",
    pretrained=False,        
    num_classes=num_classes
)

print("Завантажуємо ваги з чекпойнта...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
state_dict = torch.load(checkpoint_path, map_location=device)
model.load_state_dict(state_dict)
model.to(device)
model.eval()

print("🎉 Модель (timm ConvNeXt) успішно завантажена і готова до передбачень!")

Створення архітектури ConvNeXt (timm) для inference...
Завантажуємо ваги з чекпойнта...
🎉 Модель (timm ConvNeXt) успішно завантажена і готова до передбачень!


In [ ]:
class ButterflyTestDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.folder_path, img_name)


        image = Image.open(img_path).convert("RGB")


        if self.transform:
            image = self.transform(image)


        return image, img_name


test_dir = 'data/test'


test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


test_dataset = ButterflyTestDataset(test_dir, transform=test_transforms)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False) # shuffle=False важливо!

print(f"Знайдено тестових зображень: {len(test_dataset)}")

Знайдено тестових зображень: 2728


In [ ]:
try:
    class_names = train_dataset.classes
except NameError:
    print("Відновлюємо список класів...")
    temp_data = datasets.ImageFolder('data/train')
    class_names = temp_data.classes

print(f"Всього класів: {len(class_names)}")

all_ids = []
all_preds = []

model.eval() 
model.to(device)

print("⏳ Починаю передбачення...")

with torch.no_grad(): 
    for inputs, filenames in tqdm(test_loader):
        inputs = inputs.to(device)

        outputs = model(inputs)

        _, predicted_indices = torch.max(outputs, 1)

        for i in range(len(filenames)):
            file_id = os.path.splitext(filenames[i])[0]
            all_ids.append(file_id)

            class_idx = predicted_indices[i].item()
            predicted_label = class_names[class_idx]
            all_preds.append(predicted_label)


submission_df = pd.DataFrame({
    'id': all_ids,
    'label': all_preds
})


csv_path = 'submission_convnext_unfreeze.csv'
submission_df.to_csv(csv_path, index=False)

print(f"\n✅ Готово! Файл збережено: {csv_path}")
print("Ось перші 5 рядків для перевірки:")
print(submission_df.head())

Всього класів: 100
⏳ Починаю передбачення...


100%|██████████| 86/86 [00:50<00:00,  1.70it/s]


✅ Готово! Файл збережено: submission_convnext_unfreeze.csv
Ось перші 5 рядків для перевірки:
                                 id                   label
0  000895d0cb1745a6ababde4a4ee3b63b                 CRECENT
1  00171832b0ec4366991c749a29709211  MILBERTS TORTOISESHELL
2  00686af23e0b4c1daca1862b8c9a4e54            LARGE MARBLE
3  00763b326a404ff0b4d61076c4c19ac5           BECKERS WHITE
4  007c25ce5a8141e3a834a35c83571755            GREAT EGGFLY
